# Megamaster

In [1]:
import pandas as pd
import re
from pathlib import Path
import os
import numpy as np

pd.set_option('display.max_columns', 500)

In [2]:
from ipyfilechooser import FileChooser

fc = FileChooser("C:/Playground/bhaiya-orders/zoho-bhor-farms/inputs/")
fc.use_dir_icons = True
display(fc)

FileChooser(path='C:/Playground/bhaiya-orders/zoho-bhor-farms/inputs/', filename='', show_hidden='False')

In [3]:
FILE = Path(fc.selected)

In [4]:
OUTPUT_DIR = Path("C:/Playground/bhaiya-orders/zoho-bhor-farms/outputs/")

save_pth = Path(OUTPUT_DIR/str(FILE.stem)); save_pth
os.makedirs(save_pth, exist_ok=True)

PDF_DIR = save_pth/'customerwise-pdfs'
os.makedirs(PDF_DIR, exist_ok=True)

In [5]:
# items2subset = ['Gawar/Cluster Beans', 'Mint Leaves', 'Sponge Gourd',  'Tondli', 'VNR Guava', ]
items2subset = None
STATUSES2PROCESS = ['Completed', 'Network Error']

In [6]:
try:
    if 'csv' in FILE.suffix: df = pd.read_csv(FILE)
    else: 
        df = pd.read_excel(FILE)
        if len(pd.ExcelFile(FILE).sheet_names) > 1:
            print("WARNING: MORE THAN ONE SHEETS PRESENT IN FILE")
            
except OSError:
    print("Error occured, invalid file!")

In [7]:
df.head(3)

,Name,Select Drop Point,Select Building Ivory,Select Building Wellness Forever,Select Building NCP,Select Building Ajmera iLand,Select Building Sector 1/2,Select Building Cosmic,Select Wing,Flat No,Phone,Email,Select Delivery Date,Potatoes,Potatoes Price,Potatoes Cost Price,Select Quantity of Potatoes,Red Onions,Red Onions Price,Red Onions Cost Price,Select Quantity of Red Onions,Cauliflower,Cauliflower Price,Cauliflower Cost Price,Select Quantity of Cauliflower,Cabbage,Cabbage Price,Cabbage Cost Price,Select Quantity of Cabbage,Brinjal Bharta,Brinjal Bharta Price,Brinjal Bharta Cost Price,Select Quantity of Brinjal Bharta,Pumpkin,Pumpkin Price,Pumpkin Cost Price,Select Quantity of Pumpkin,Brinjal Kateri,Brinjal Kateri Price,Brinjal Kateri Cost Price,Select Quantity of Brinjal Kateri,Cucumber,Cucumber Price,Cucumber Cost Price,Select Quantity of Cucumber,Bhindi/Lady's Finger,Bhindi/Lady's Finger Price,Bhindi/Lady's Finger Cost Price,Select Quantity of Bhindi/Lady's Finger,Bitter Gourd,Bitter Gourd Price,Bitter Gourd Cost Price,Select Quantity of Bitter Gourd,Ridge Gourd,Ridge Gourd Price,Ridge Gourd Cost Price,Select Quantity of Ridge Gourd,Beetroot,Beetroot Price,Beetroot Cost Price,Select Quantity of Beetroot,Button Mushrooms,Button Mushroom Price,Button Mushroom Cost Price,Select Quantity of Button Mushroom,Bottle Gourd,Bottle Gourd Price,Bottle Gourd Cost Price,Select Quantity of Bottle Gourd,Drumsticks,Drumsticks Price,Drumsticks Cost Price,Select Quantity of Drumsticks,Capsicum,Capsicum Price,Capsicum Cost Price,Select Quantity of Capsicum,Gawar/Cluster Beans,Gawar/Cluster Beans Price,Gawar/Cluster Beans Cost Price,Select Quantity of Gawar/Cluster Beans,French Beans,French Beans Price,French Beans Cost Price,Select Quantity of French Beans,Tomato,Tomato Price,Tomato Cost Price,Select Quantity of Tomato,Sweet Corn,Sweet Corn Price,Sweet Corn Cost Price,Select Quantity of Sweet Corn,Broccoli,Broccoli Price,Broccoli Cost Price,Select Quantity of Broccoli,Lettuce,Lettuce Price,Lettuce Cost Price,Select Quantity of Lettuce,Kiwi Zespri Imported,Kiwi Zespri Imported Price,Kiwi Zespri Imported Cost Price,Select Quantity of Kiwi Zespri Imported,Banana Regular,Banana Price,Banana Cost Price,Select Quantity of Banana,Garlic,Garlic Price,Garlic Cost Price,Select Quantity of Garlic,Ginger,Ginger Price,Ginger Cost Price,Select Quantity of Ginger,Green Chilly,Green Chilly Price,Green Chilly Cost Price,Select Quantity of Green Chilly,Lemons,Lemons Price,Lemons Cost Price,Select Quantity of Lemons,Curry Leaves,Curry Leaves Price,Curry Leaves Cost Price,Select Quantity of Curry Leaves,Coriander Leaves,Coriander Leaves Price,Coriander Leaves Cost Price,Select Quantity of Coriander Leaves,Mint Leaves,Mint Leaves Price,Mint Leaves Cost Price,Select Quantity of Mint Leaves,Palak/Spinach,Palak/Spinach Price,Palak/Spinach Cost Price,Select Quantity of Palak/Spinach,Shepu/Dill Leaves,Shepu/Dill Leaves Price,Shepu/Dill Leaves Cost Price,Select Quantity of Shepu/Dill Leaves,Methi Leaves,Methi Leaves Price,Methi Leaves Cost Price,Select Quantity of Methi Leaves,Red/Yellow Bell Peppers,Red/Yellow Bell Peppers Price,Red/Yellow Bell Peppers Cost Price,Select Quantity of Red/Yellow Bell Peppers,Carrots,Carrots Price,Carrots Cost Price,Select Quantity of Carrots,Radish/Muli,Radish/Muli Price,Radish/Muli Cost Price,Select Quantity of Radish/Muli,Green Peas,Green Peas Price,Green Peas Cost Price,Select Quantity of Green Peas,Lal Math/Amaranthus Leaves,Lal Math/Amaranthus Price,Lal Math/Amaranthus Cost Price,Select Quantity of Lal Math/Amaranthus,Total Amount,Payment Amount,Payment Status,Payment Currency,Added Time,CRM Status,Referrer Name,Task Owner
0,anjana; dubey,Lodha NCP,NaN,NaN,Tower 6,NaN,NaN,NaN,B Wing,3004,7739401733,dubey.anjana81@gmail.com,Thu Aug 27th,NaN,40,34,2.0,NaN,23,17,2.0,NaN,45,29,NaN,NaN,30,19,1.0,NaN,32,19,NaN,NaN,20,11,NaN,NaN,35,21,NaN,NaN,24,11,3.0,NaN,30,18,1.0,NaN,20,8,NaN,NaN,50,30,1.0,NaN,20,13,NaN,NaN,60,38,NaN,NaN,30,13,1.0,

## Add indexing key

In [8]:
df['key'] = df['Name'] + ' ' + df['Added Time'].astype(str)

## Pull out the Select Building columns

In [9]:
bldg_cols = []

for col in df.columns:
    if 'select building' in col.lower():
        bldg_cols.append(col)

df['building'] = df[bldg_cols].fillna('').sum(axis=1)

## Subset only successfully paid orders

In [10]:
df['Payment Status'].unique()

array(['Completed', 'Network Error'], dtype=object)

In [11]:
df = df[df['Payment Status'].isin(STATUSES2PROCESS)]

In [12]:
# verification
df['Payment Status'].unique()

array(['Completed', 'Network Error'], dtype=object)

## Generating Tokens

Sort by Drop Point, Building, Wing and then generate Token Values.

In [13]:
df = df.sort_values(['Select Drop Point', 'building', 'Select Wing'])
df['Token No'] = range(1, len(df)+1)

## Check for duplicate entries

In [14]:
assert len(df) == len(df['key'].unique())

## Build Items Dictionary

    orders dictionary, one record looks like:

    {
        key:
        {
            name : str,
            drop_point : str,
            building : str,
            wing : str,
            flat : str,
            phone : numeric,
            email : str,
            delivery_date : datetime,
            order : dict,
            total_amount : float,
            payment_status : str
        }
    }


    items_dict:

    {
        item_name : 
        {
            sell_price : float,
            cost_price : float,
            packing_size : // to be read from another excel
        }
    }

If 'quantity of' occurs in column header, leave the first three words and extract the rest as your item key.

In [15]:
# extremely naive, needs a fix
all_items = [" ".join(i.split()[3:]) for i in df.columns if 'quantity of' in i.lower()]

In [16]:
items_dict = {}

In [17]:
def get_sell_price(item):
    val = df[item + ' ' + 'Price'].values[0]
    return val

def get_cost_price(item):
    val = df[item + ' ' + 'Cost Price'].values[0]
    return val

In [18]:
df_psize = pd.read_csv("C:/Playground/bhaiya-orders/zoho-bhor-farms/database/Packing Size.csv")
psize_dict = df_psize.set_index('Item').to_dict(orient='index')

In [19]:
def get_packing_size(item):
#     if item in psize_dict.keys():
    return psize_dict[item]['Size']
#     return 1

def get_unit(item):
#     if item in psize_dict.keys():
    return psize_dict[item]['Unit']
#     return 1

In [20]:
for item in all_items:
    dct = {}
    dct['packing_size'] = get_packing_size(item)
    dct['unit'] = get_unit(item)
#     if item == 'Cherry Tomatoes': item = 'Cherry Tomato'
#     if item == 'Potatoes': item = 'Potato'
    
    dct['sell_price'] = get_sell_price(item)
    dct['cost_price'] = get_cost_price(item)
    items_dict[item] = dct

## Build Master Dictionary

In [21]:
# # because few flats were not captured
df['Flat No'] = df['Flat No'].fillna('NOT CAPTURED')

In [22]:
df.fillna(0, inplace=True)

In [23]:
# create a mapping from item_id to the corresponding column to make sure no item goes unmapped

item2dropdowncol = {i : '' for i in all_items}

for index, row in df.iterrows():
    for col in df.columns:
        if 'quantity of' in col.lower():
            item2dropdowncol[" ".join(col.split()[3:])] = col
            
assert len(item2dropdowncol) == len(all_items)
for k, v in item2dropdowncol.items():
    assert len(v) > 3


    {
        key:
        {
            name : str,
            drop_point : str,
            building : str,
            wing : str,
            flat : str,
            phone : numeric,
            email : str,
            delivery_date : datetime,
            order : dict,
            total_amount : float,
            payment_status : str
        }
    }

In [24]:
def get_order(row):
    o = {}
    for item, qty_col in item2dropdowncol.items():
#         try:
        if row[qty_col] > 0:
            o[item] = row[qty_col]
#         except:
#             i = 'hello'
            
    return o

In [25]:
def get_total_cost_price(order):
    totalcp = 0
    for item, qty in order.items():
#         if item == 'Cherry Tomatoes': item = 'Cherry Tomato'
#         if item == 'Potatoes': item = 'Potato'
        totalcp += qty * get_cost_price(item)
    return totalcp

In [26]:
orders_dict = {}

for index, row in df.iterrows():
    dct = {}
    dct['name'] = row['Name']
    dct['drop_point'] = row['Select Drop Point']
    dct['building'] = row['building']
    dct['wing'] = row['Select Wing']
    dct['flat_no'] = row['Flat No']
    dct['phone_no'] = row['Phone']
    dct['email'] = row['Email']
    dct['delivery_date'] = row['Select Delivery Date']
    dct['total_amount'] = row['Total Amount']
    dct['payment_status'] = row['Payment Status']
    dct['order'] = get_order(row)
    dct['total_cost_price'] = get_total_cost_price(dct['order'])
    dct['profit'] = dct['total_amount'] - dct['total_cost_price']
    orders_dict[row['Token No']] = dct

# Subset Items : To Process Refunds

In [27]:
# execute only if items2subset > 0

In [28]:
rows_list = []

for token, o in orders_dict.items():
    for item in items2subset:
        if item in o['order'].keys():
            dct = {}
            dct['name'] = o['name']
            dct['phone_no'] = o['phone_no']
            dct['email'] = o['email']
            dct['item_name'] = item
            dct['item_qty'] = o['order'][item]
            dct['item_price_per_unit'] = items_dict[item]['sell_price']
            dct['item_amount'] = items_dict[item]['sell_price'] * dct['item_qty']
            dct['amount_paid'] = o['total_amount']
            dct['payment_status'] = o['payment_status']
            
            rows_list.append(dct)

TypeError: 'NoneType' object is not iterable

In [89]:
subset_df = pd.DataFrame(rows_list)

In [56]:
amount2refund = subset_df.groupby('name').sum()['item_amount'].to_dict()

In [90]:
subset_df['amount_to_refund'] = subset_df.name.apply(lambda x: amount2refund[x])

In [93]:
fname = save_pth/('unavl-'+FILE.stem+'.xlsx')

In [94]:
with pd.ExcelWriter(fname) as writer:
    subset_df.to_excel(writer, sheet_name='itemwise')
    (subset_df.groupby(['name', 'phone_no'])
        .first()[['email', 'amount_paid', 'amount_to_refund']]
        .to_excel(writer, sheet_name='refund_amount_agg'))

# Create Listified File

In [28]:
dict_listified = {i : 0 for i in all_items}

In [29]:
for key, value in orders_dict.items():
    for item, qty in value['order'].items():
        dict_listified[item] += qty

In [30]:
df_listified = pd.DataFrame()

df_listified['item'] = all_items
df_listified['packing_size'] = df_listified['item'].apply(lambda x: get_packing_size(x))
df_listified['num_units'] = df_listified['item'].apply(lambda x: dict_listified[x])
df_listified['weight'] = df_listified.packing_size * df_listified.num_units
df_listified['unit'] = df_listified['item'].apply(lambda x: get_unit(x))

In [31]:
df_listified = df_listified[df_listified.num_units > 0]; df_listified

,item,packing_size,num_units,weight,unit
0,Potatoes,1.00,65.0,65.00,Kg
1,Red Onions,1.00,78.0,78.00,Kg
2,Cauliflower,1.00,41.0,41.00,Pieces
3,Cabbage,1.00,19.0,19.00,Pieces
4,Brinjal Bharta,1.00,28.0,28.00,Pieces
5,Pumpkin,0.50,19.0,9.50,Kg
7,Cucumber,0.50,64.0,32.00,Kg
8,Bhindi/Lady's Finger,0.50,53.0,26.50,Kg
9,Bitter Gourd,0.30,28.0,8.40,Kg
10,Ridge Gourd,0.50,33.0,16.50,Kg


In [32]:
fname = save_pth/('quantity-' + str(FILE.stem) + '.csv')

In [33]:
df_listified[df_listified.num_units>0].to_csv(fname, index=False)

# Customerwise

In [34]:
items = ['coconut', 'kiwi']

In [35]:
df_trans = pd.read_excel("C:/Playground/bhaiya-orders/zoho-bhor-farms/database/translations.xlsx")

In [36]:
from fuzzywuzzy import fuzz

def get_match(x, thresh=65):
    x = x.lower()
    rmax = -1
    for k in items: 
        if fuzz.partial_ratio(k, x) > rmax:
            rmax = fuzz.partial_ratio(k, x)

    return rmax > thresh

In [37]:
keys2drop = df_trans[df_trans['Final Item Name'].apply(get_match)]['Final Item Name'].values; keys2drop

array(['Coconut', 'Kiwi Imported', 'Coconut Mangalore Small',
       'Kiwi Imported', ' Coconut Mangalore Big',
       ' Coconut Mangalore Medium', 'Kiwi Imported - Zespri Green'],
      dtype=object)

In [38]:
df_trans = df_trans.set_index('Final Item Name').drop(keys2drop).reset_index()
eng2marathi = df_trans.set_index('Final Item Name').to_dict()['Final Marathi Name']
marathi2eng = df_trans.set_index('Final Marathi Name').to_dict()['Final Item Name']
marathi2eng[np.nan] = ''

In [39]:
def get_trans(x, thresh=85):
    rmax = -1
    trans = ''
    for k in eng2marathi.keys():
        if fuzz.ratio(k, x) > rmax:
            rmax = fuzz.ratio(k, x)
            trans = eng2marathi[k]

    if rmax > thresh:
        return trans
    return np.nan

In [40]:
rows_list = []
for token, o in orders_dict.items():
    for item, qty in o['order'].items():
        row_dct = {}
        row_dct['token_no'] = token
        row_dct['name'] = o['name']
        row_dct['delivery_date'] = o['delivery_date']
        row_dct['item_name'] = item
        row_dct['marathi_name'] = get_trans(item)
        row_dct['num_packets'] = qty
        row_dct['packing_size'] = get_packing_size(item)
        row_dct['final_weight'] = row_dct['num_packets'] * row_dct['packing_size']
        row_dct['unit'] = get_unit(item)
        row_dct['total_amt'] = o['total_amount']
        
        rows_list.append(row_dct)

In [41]:
df_customerwise = pd.DataFrame(rows_list)

In [42]:
df_customerwise.head()

,token_no,name,delivery_date,item_name,marathi_name,num_packets,packing_size,final_weight,unit,total_amt
0,1,Rashmita; Sundaram,Thu Aug 27th,Bitter Gourd,कारले,2.0,0.30,0.60,Kg,257
1,1,Rashmita; Sundaram,Thu Aug 27th,Ridge Gourd,दोडका,1.0,0.50,0.50,Kg,257
2,1,Rashmita; Sundaram,Thu Aug 27th,Drumsticks,शेवगा,1.0,0.25,0.25,Kg,257
3,1,Rashmita; Sundaram,Thu Aug 27th,Banana,केळी,1.0,6.00,6.00,Pieces,257
4,1,Rashmita; Sundaram,Thu Aug 27th,Green Chilly,हिरवी मिरची,1.0,0.20,0.20,kg,257


In [43]:
notranslist = df_customerwise[df_customerwise.marathi_name.isna()].item_name.unique(); notranslist

array(['Kiwi Zespri Imported'], dtype=object)

## Kiwi, Coconut Alag Se

In [44]:
ids = set([i.lower()[:4] for i in notranslist])
prefix = "".join(ids)

notrans_pth = save_pth/f'{prefix}-{str(FILE.stem)}.csv'; notrans_pth.stem

'kiwi-27th Aug Vegetables Final'

In [45]:
(df_customerwise[df_customerwise.marathi_name.isna()]
    .drop(columns=['marathi_name', 'total_amt'])
    .to_csv(notrans_pth, index=False))

## Drop items whose translations are absent from the Customerwise File

In [46]:
df_customerwise.marathi_name = df_customerwise.marathi_name.fillna('')

In [47]:
# df_customerwise.dropna(subset=['marathi_name'], inplace=True)

## Build Excel Savefile

In [48]:
highlight_col = 'num_packets'

In [49]:
custs = df_customerwise.name + '_' + df_customerwise.token_no.astype(str)
custs = custs.unique()

In [50]:
def f(x):
    col = highlight_col
    r = 'background-color: none'
    g = 'background-color: #37FDFC'
    c = np.where(x[col] > 1, g, r)
    y = pd.DataFrame('', index=x.index, columns=x.columns)
    for h in all_cols:
        y[h] = c
    
    return y

In [51]:
def boldify(x):
    col = 'total_amt'
    n = 'font-weight: normal'
    b = 'font-weight: bold'
    c = np.where(x[col] >= 1000.0, b, n)
    y = pd.DataFrame('', index=x.index, columns=x.columns)
    for h in all_cols:
        y[h] = c
    
    return y

In [52]:
all_cols = df_customerwise.columns

In [53]:
def get_sum_packets(token):
    return sum(orders_dict[token]['order'].values())

In [54]:
fname = save_pth/('customerwise-' + str(FILE.stem) + '.xlsx')

In [55]:
all_data = 0

with pd.ExcelWriter(fname) as writer:
    for cust in custs:
        data = df_customerwise[df_customerwise.name == cust.split('_')[0]]
        data = data[data.token_no == int(cust.split('_')[1])]
        all_data += len(data.index)

        data = data.append({'marathi_name': 'TOTAL PACKETS', 'num_packets': get_sum_packets(
            int(cust.split('_')[1]))}, ignore_index=True)
        styled = data.style.apply(f, axis=None)
        styled = styled.apply(boldify, axis=None)
        styled.to_excel(writer, columns=['token_no', 'name', 'delivery_date', 'item_name', 'marathi_name',
                                         'num_packets', 'packing_size', 'final_weight', 'unit'], sheet_name=cust[:31], engine='openpyxl', index=False)

assert all_data == len(df_customerwise)

## Build PDF Savefile

In [56]:
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfbase.ttfonts import TTFont  
from reportlab.pdfbase import pdfmetrics
from reportlab.platypus import Table, TableStyle, PageBreak, SimpleDocTemplate
from reportlab.lib import colors

pdfmetrics.registerFont(TTFont("mangal", "font/Lohit-Devanagari.ttf"))

In [57]:
df_customerwise.drop(columns=['delivery_date', 'total_amt'], inplace=True)

In [58]:
# get index of num_packets, marathi_name
for i, col in enumerate(df_customerwise.columns): 
    if col == highlight_col: qty_index = i
    if col == 'marathi_name': marathi_index = i

In [59]:
def create_pdf(data):
    
    pdfy_data = data.to_numpy().tolist()
    pdfy_data.insert(0, data.columns.values.tolist())

#     pdf = SimpleDocTemplate(
#         fname,
#         pagesize=landscape(letter),
#     #     font='mangal'
#     )

    table = Table(pdfy_data)

    style = TableStyle([
    #     ('BACKGROUND', (0,0), (3,0), colors.green),
    #     ('TEXTCOLOR',(0,0),(-1,0),colors.whitesmoke),

        ('ALIGN',(0,0),(-1,-1),'CENTER'),
    #     ('FONTNAME', (0,0), (-1,0), 'Courier-Bold'),
        ('FONTSIZE', (0,0), (-1,0), 10),
        ('BOTTOMPADDING', (0,0), (-1,0), 12),
        ('FONTNAME',(marathi_index,1),(marathi_index,-1),'mangal'),
        ('BACKGROUND',(0,0),(-1,0),colors.burlywood),
        ('BACKGROUND',(qty_index,-1),(qty_index,-1),colors.springgreen)
    ])
    
    table.setStyle(style)

    rowNumb = len(pdfy_data)
    for i in range(1, rowNumb-1):
        if pdfy_data[i][qty_index] > 1:
            bc = colors.aqua
        else:
            bc = colors.white
        ts = TableStyle(
            [('BACKGROUND', (0,i),(-1,i), bc)]
        )
        table.setStyle(ts)

    # 3) Add borders
    ts = TableStyle(
        [
    #     ('BOX',(0,0),(-1,-1),2,colors.black),

    #     ('LINEBEFORE',(2,1),(2,-1),2,colors.red),
    #     ('LINEABOVE',(0,2),(-1,2),2,colors.green),

        ('GRID',(0,0),(-1,-2),2,colors.black),
        ]
    )
    table.setStyle(ts)

#     elems = []
    elems.append(table)
    elems.append(PageBreak())
    
#     pdf.build(elems)

In [60]:
fname = str(PDF_DIR/('customerwise-' + str(FILE.stem) + '.pdf'))

In [61]:
pdf = SimpleDocTemplate(
        fname,
        pagesize=landscape(letter),
    #     font='mangal'
    )

elems = []

In [62]:
all_data = 0

for cust in custs:
    data = df_customerwise[df_customerwise.name == cust.split('_')[0]]
    data = data[data.token_no == int(cust.split('_')[1])]
    
    data.token_no = data.token_no.astype(int)
    data.num_packets = data.num_packets.astype(int)
    
    all_data += len(data.index)
    
    data = data.append({'num_packets': get_sum_packets(
        int(cust.split('_')[1]))}, ignore_index=True)

    data = data.fillna('')
    create_pdf(data)

assert all_data == len(df_customerwise)

In [63]:
pdf.build(elems)

# Printout File

In [64]:
# decision for vegetables vs seasonal fruits

In [65]:
def is_trans(x, thresh=85):
    rmax = -1
    trans = ''
    for k in eng2marathi.keys():
        if fuzz.ratio(k, x) > rmax:
            rmax = fuzz.ratio(k, x)
            trans = eng2marathi[k]

    return rmax > thresh

In [66]:
translations = [is_trans(i) for i in all_items]

In [67]:
pipe_separated = sum(translations)/len(translations) < 0.4 # less than 40% translations are present

In [68]:
pipe_separated

False

## Sudeep

In [69]:
def get_pipe_separated(token):
    order = orders_dict[token]['order']
    s=[]
    for k, v in order.items():
        s.append(k + ': ' + str(int(v)))
    
    return ' | '.join(s)

In [70]:
rows_list_sudeep = []
for token, o in orders_dict.items():
    row_dct = {}
    row_dct['token_no'] = token
    row_dct['name'] = o['name']
    row_dct['drop_point'] = o['drop_point']
    row_dct['building'] = o['building']
    row_dct['wing'] = o['wing']
    row_dct['flat_no'] = o['flat_no']
    row_dct['phone_no'] = o['phone_no']
    row_dct['email'] = o['email']
    
    if pipe_separated: row_dct['items'] = get_pipe_separated(token)
    else: row_dct['num_packets'] = get_sum_packets(token)
    
    row_dct['total_amount'] = o['total_amount']
    row_dct['total_cp'] = o['total_cost_price']
    row_dct['profit'] = o['profit']
    
    
    rows_list_sudeep.append(row_dct)

In [71]:
df_print_sudeep = pd.DataFrame(rows_list_sudeep)

In [72]:
df_print_sudeep.head()

,token_no,name,drop_point,building,wing,flat_no,phone_no,email,num_packets,total_amount,total_cp,profit
0,1,Rashmita; Sundaram,Ajmera iLand,Aeon,A Wing,1202,9004913659,Rashmita.sundaram@gmail.com,11.0,257,143.0,114.0
1,2,Kumkum; Jain,Ajmera iLand,Aeon,A Wing,302,9930160637,Jainkmkm@gmail.com,20.0,635,378.0,257.0
2,3,Preety; Agarwal,Ajmera iLand,Aeon,B Wing,803,9921001132,preetyanil@yahoo.co.in,11.0,480,314.0,166.0
3,4,Madhavi; Ravuri,Ajmera iLand,Zeon,A Wing,803,9820157581,ravurimadhavi@ymail.com,21.0,778,484.0,294.0
4,5,Minakshi; Jaiswal,Ajmera iLand,Zeon,A Wing,604,9867041235,minakshi.jaiswal09@gmail.com,12.0,375,244.0,131.0


In [73]:
save_pth

WindowsPath('C:/Playground/bhaiya-orders/zoho-bhor-farms/outputs/27th Aug Vegetables Final')

In [74]:
df_print_sudeep.to_csv(save_pth/'print-sudeep.csv', index=False)

## Vishal

In [75]:
rows_list_vishal = []
for token, o in orders_dict.items():
    row_dct = {}
    row_dct['token_no'] = token
    row_dct['name'] = o['name']
    row_dct['drop_point'] = o['drop_point']
    row_dct['building'] = o['building']
    row_dct['wing'] = o['wing']
    row_dct['flat_no'] = o['flat_no']
#     row_dct['phone_no'] = o['phone_no']
#     row_dct['email'] = o['email']

    if pipe_separated: row_dct['items'] = get_pipe_separated(token)
    else: row_dct['num_packets'] = get_sum_packets(token)    
    
    if 'total_amount' in o.keys():
        row_dct['total_amount'] = o['total_amount']
    
    rows_list_vishal.append(row_dct)

In [76]:
df_print_vishal = pd.DataFrame(rows_list_vishal)

In [77]:
df_print_vishal.head()

,token_no,name,drop_point,building,wing,flat_no,num_packets,total_amount
0,1,Rashmita; Sundaram,Ajmera iLand,Aeon,A Wing,1202,11.0,257
1,2,Kumkum; Jain,Ajmera iLand,Aeon,A Wing,302,20.0,635
2,3,Preety; Agarwal,Ajmera iLand,Aeon,B Wing,803,11.0,480
3,4,Madhavi; Ravuri,Ajmera iLand,Zeon,A Wing,803,21.0,778
4,5,Minakshi; Jaiswal,Ajmera iLand,Zeon,A Wing,604,12.0,375


In [78]:
df_print_vishal.to_csv(save_pth/'print-vishal.csv', index=False)

## Dropwise 

In [79]:
drops = df_print_vishal.drop_point.unique()

In [80]:
# fname = save_pth/"dropwise-phool.xlsx"
fname = save_pth/('dropwise-' + str(FILE.stem) + '.xlsx')

In [81]:
all_data = 0

with pd.ExcelWriter(fname) as writer:
    for drop in drops:
        data = df_print_vishal[df_print_vishal.drop_point == drop]
        all_data += len(data.index)

        invalid_chars = '[]:*?/\\'
        for c in invalid_chars: drop = drop.replace(c, '')
        
        data.to_excel(writer, sheet_name=drop, index=False)
    
assert all_data == len(df_print_vishal)

## Tokens

In [82]:
subset_cols = ['name', 'token_no', 'building', 'wing', 'flat_no']

In [83]:
df_tokens = df_print_vishal[subset_cols]

In [84]:
drops = df_print_vishal.drop_point.unique()

In [85]:
# fname = save_pth/"tokens-phool.xlsx"
fname = save_pth/('tokens-' + str(FILE.stem) + '.xlsx')

In [86]:
all_data = 0

with pd.ExcelWriter(fname) as writer:
    for drop in drops:
        data = df_print_vishal[df_print_vishal.drop_point == drop]
        all_data += len(data.index)
        
        invalid_chars = '[]:*?/\\'
        for c in invalid_chars: drop = drop.replace(c, '')
        
        data.to_excel(writer, sheet_name=drop, columns=subset_cols, index=False)
    
assert all_data == len(df_print_vishal)

# Dropwise Count

In [87]:
rows_list = []
for token, order in orders_dict.items():
    for item, count in order['order'].items():
        row_dct = {}
        row_dct['drop_point'] = order['drop_point']
        row_dct['item_name'] = item
        row_dct['num_packets'] = count
        
        rows_list.append(row_dct)

In [88]:
df_dropwise = pd.DataFrame(rows_list)

In [89]:
df_dropwise

,drop_point,item_name,num_packets
0,Ajmera iLand,Bitter Gourd,2.0
1,Ajmera iLand,Ridge Gourd,1.0
2,Ajmera iLand,Drumsticks,1.0
3,Ajmera iLand,Banana,1.0
4,Ajmera iLand,Green Chilly,1.0
...,...,...,...
1107,Sector 1/2,French Beans,1.0
1108,Sector 1/2,Tomato,3.0
1109,Sector 1/2,Sweet Corn,2.0
1110,Sector 1/2,Lemons,1.0


In [90]:
fname = save_pth/('dropwisecount-' + str(FILE.stem) + '.csv')

In [91]:
df_dropwise.groupby(['drop_point', 'item_name']).sum().reset_index().to_csv(fname, index=False)

# End

## Manually Adding Orders

In [56]:
for a in all_items: print(a)

Haar 1
Haar 2
Jhendu/Marigold
Druva
Tulsi
Pack of 5 leaves
Betelnut
Banana Leaves with Stem
Pack of 5 Fruits
Assorted Flowers
Hibiscus


In [57]:
new_orders = pd.read_csv('yelofile/additions.csv')

In [58]:
keys1 = ['name', 'drop_point', 'building', 'wing', 'flat_no', 'phone_no', 'email', 'delivery_date', 'total_amount', 'order']

In [59]:
new_orders.dropna(inplace=True)

In [60]:
new_orders

,index,Haar 1,Haar 2,Jhendu/Marigold,Rajnigandha,Druva,Tulsi,Pack of 5 leaves,Betelnut,Banana Leaves with Stem,Banana Leaves,Pack of 5 Fruits,Assorted Flowers,Hibiscus


In [133]:
to_append = new_orders.to_dict(orient='records')

In [134]:
orders_very_very_new = pd.read_csv('yelofile/additions.csv')
orders_very_very_new.set_index('index', inplace=True)
orders_very_very_new.fillna(0, inplace=True)

In [137]:
for i in to_append:
    i['order'] = new_items[i['name']]

In [61]:
to_append = [{'name': 'Suchitra;Narale',
  'drop_point': 'Cosmic',
  'building': 'Cosmic Heights',
  'wing': 'B Wing',
  'flat_no': 2603.0,
  'phone_no': 9833567340.0,
  'email': 'suchitrahate@gmail.com',
  'order': {'Haar 1': 1.0,
   'Haar 2': 0.0,
   'Jhendu/Marigold': 1.0,
   'Rajnigandha': 0.0,
   'Druva': 1.0,
   'Tulsi': 0.0,
   'Pack of 5 leaves': 0.0,
   'Betelnut': 0.0,
   'Banana Leaves with Stem': 0.0,
   'Banana Leaves': 0.0,
   'Pack of 5 Fruits': 1.0,
   'Assorted Flowers': 1,
   'Hibiscus': 0.0}},
 {'name': 'Ritu;Sharda',
  'drop_point': 'Cosmic',
  'building': 'Cosmic Heights',
  'wing': 'B Wing',
  'flat_no': 2101.0,
  'phone_no': 9820844319.0,
  'email': 'shardaritu@hotmail.com',
  'order': {'Haar 1': 0.0,
   'Haar 2': 0.0,
   'Jhendu/Marigold': 2.0,
   'Rajnigandha': 0.0,
   'Druva': 1.0,
   'Tulsi': 0.0,
   'Pack of 5 leaves': 0.0,
   'Betelnut': 1.0,
   'Banana Leaves with Stem': 0.0,
   'Banana Leaves': 1.0,
   'Pack of 5 Fruits': 0.0,
   'Assorted Flowers': 1,
   'Hibiscus': 0.0}},
 {'name': 'Nisha;Narale',
  'drop_point': 'Cosmic',
  'building': 'Cosmic Heights',
  'wing': 'A Wing',
  'flat_no': 2505.0,
  'phone_no': 9930390994.0,
  'email': 'nishashetti@gmail.com',
  'order': {'Haar 1': 1.0,
   'Haar 2': 1.0,
   'Jhendu/Marigold': 0.0,
   'Rajnigandha': 0.0,
   'Druva': 1.0,
   'Tulsi': 1.0,
   'Pack of 5 leaves': 0.0,
   'Betelnut': 1.0,
   'Banana Leaves with Stem': 0.0,
   'Banana Leaves': 1.0,
   'Pack of 5 Fruits': 2.0,
   'Assorted Flowers': 3,
   'Hibiscus': 1.0}},
 {'name': 'Padmaja;Kakkera',
  'drop_point': 'Lodha NCP',
  'building': 'Tower 4',
  'wing': 'B Wing',
  'flat_no': 202.0,
  'phone_no': 9900065600.0,
  'email': 'padmaja.kakkera@gmail.com',
  'order': {'Haar 1': 0.0,
   'Haar 2': 0.0,
   'Jhendu/Marigold': 0.0,
   'Rajnigandha': 0.0,
   'Druva': 1.0,
   'Tulsi': 1.0,
   'Pack of 5 leaves': 1.0,
   'Betelnut': 1.0,
   'Banana Leaves with Stem': 0.0,
   'Banana Leaves': 0.0,
   'Pack of 5 Fruits': 1.0,
   'Assorted Flowers': 1,
   'Hibiscus': 1.0}},
 {'name': 'Shalini;Memani',
  'drop_point': 'Lodha NCP',
  'building': 'Tower 6',
  'wing': 'A Wing',
  'flat_no': 2503.0,
  'phone_no': 9323134454.0,
  'email': 'shalini_depura@yahoo.co.in',
  'order': {'Haar 1': 2.0,
   'Haar 2': 0.0,
   'Jhendu/Marigold': 0.0,
   'Rajnigandha': 0.0,
   'Druva': 2.0,
   'Tulsi': 0.0,
   'Pack of 5 leaves': 0.0,
   'Betelnut': 1.0,
   'Banana Leaves with Stem': 0.0,
   'Banana Leaves': 2.0,
   'Pack of 5 Fruits': 0.0,
   'Assorted Flowers': 1,
   'Hibiscus': 3.0}},
 {'name': 'Ila Panchakshari',
  'drop_point': 'Ivory',
  'building': 'Himalayan Heights',
  'wing': 'A Wing',
  'flat_no': 1003.0,
  'phone_no': 8286125928.0,
  'email': 'ilahp78@gmail.com',
  'order': {'Haar 1': 0.0,
   'Haar 2': 0.0,
   'Jhendu/Marigold': 0.0,
   'Rajnigandha': 0.0,
   'Druva': 5.0,
   'Tulsi': 0.0,
   'Pack of 5 leaves': 0.0,
   'Betelnut': 1.0,
   'Banana Leaves with Stem': 0.0,
   'Banana Leaves': 0.0,
   'Pack of 5 Fruits': 0.0,
   'Assorted Flowers': 1,
   'Hibiscus': 1.0}},
 {'name': 'Swati;Madhukar',
  'drop_point': 'Lodha NCP',
  'building': 'Tower 6',
  'wing': 'A Wing',
  'flat_no': 4002.0,
  'phone_no': 9167853838.0,
  'email': 'swati.chaudhary90@yahoo.com',
  'order': {'Haar 1': 8.0,
   'Haar 2': 2.0,
   'Jhendu/Marigold': 0.0,
   'Rajnigandha': 0.0,
   'Druva': 0.0,
   'Tulsi': 0.0,
   'Pack of 5 leaves': 0.0,
   'Betelnut': 5.0,
   'Banana Leaves with Stem': 0.0,
   'Banana Leaves': 0.0,
   'Pack of 5 Fruits': 0.0,
   'Assorted Flowers': 1,
   'Hibiscus': 0.0}}]

In [68]:
new_orders_dict = {}
i = 178

for row in to_append:
    dct = {}
    dct['name'] = row['name']
#     dct['drop_point'] = row['Select Drop Point']
#     dct['building'] = row['building']
#     dct['wing'] = row['Select Wing']
#     dct['flat_no'] = row['Flat No']
    dct['phone_no'] = row['phone_no']
    dct['email'] = row['email']
#     dct['delivery_date'] = row['Select Delivery Date']
#     dct['total_amount'] = row['Total Amount']
#     dct['total_amount'] = 
#     dct['payment_status'] = row['Payment Status']
    dct['order'] = get_order(row)
    dct['total_amount'] = get_total_amt(dct['order'])
    
    new_orders_dict[i] = dct
    i+=1

In [70]:
60+100+15+130+121

426

In [71]:
480+200+25+50+121

876

In [73]:
120+30+25+121

296

In [80]:
items_dict

{'Haar 1': {'packing_size': 1, 'unit': 1, 'sell_price': 60},
 'Haar 2': {'packing_size': 1, 'unit': 1, 'sell_price': 100},
 'Jhendu/Marigold': {'packing_size': 1, 'unit': 1, 'sell_price': 100},
 'Druva': {'packing_size': 1, 'unit': 1, 'sell_price': 15},
 'Tulsi': {'packing_size': 1, 'unit': 1, 'sell_price': 20},
 'Pack of 5 leaves': {'packing_size': 1, 'unit': 1, 'sell_price': 25},
 'Betelnut': {'packing_size': 1, 'unit': 1, 'sell_price': 25},
 'Banana Leaves with Stem': {'packing_size': 1, 'unit': 1, 'sell_price': 100},
 'Pack of 5 Fruits': {'packing_size': 1, 'unit': 1, 'sell_price': 130},
 'Assorted Flowers': {'packing_size': 1, 'unit': 1, 'sell_price': 121},
 'Hibiscus': {'packing_size': 1, 'unit': 1, 'sell_price': 20}}

In [138]:
to_append

[{'name': 'Suchitra;Narale',
  'drop_point': 'Cosmic',
  'building': 'Cosmic Heights',
  'wing': 'B Wing',
  'flat_no': 2603.0,
  'phone_no': 9833567340.0,
  'email': 'suchitrahate@gmail.com',
  'order': {'Haar 1': 1.0,
   'Haar 2': 0.0,
   'Jhendu/Marigold': 1.0,
   'Rajnigandha': 0.0,
   'Druva': 1.0,
   'Tulsi': 0.0,
   'Pack of 5 leaves': 0.0,
   'Betelnut': 0.0,
   'Banana Leaves with Stem': 0.0,
   'Banana Leaves': 0.0,
   'Pack of 5 Fruits': 1.0,
   'Assorted Flowers': 1,
   'Hibiscus': 0.0}},
 {'name': 'Ritu;Sharda',
  'drop_point': 'Cosmic',
  'building': 'Cosmic Heights',
  'wing': 'B Wing',
  'flat_no': 2101.0,
  'phone_no': 9820844319.0,
  'email': 'shardaritu@hotmail.com',
  'order': {'Haar 1': 0.0,
   'Haar 2': 0.0,
   'Jhendu/Marigold': 2.0,
   'Rajnigandha': 0.0,
   'Druva': 1.0,
   'Tulsi': 0.0,
   'Pack of 5 leaves': 0.0,
   'Betelnut': 1.0,
   'Banana Leaves with Stem': 0.0,
   'Banana Leaves': 1.0,
   'Pack of 5 Fruits': 0.0,
   'Assorted Flowers': 1,
   'Hibiscus':

In [ ]:
for k in to_a

In [136]:
new_items = orders_very_very_new.to_dict(orient='index')

In [74]:
keys2 = ['Haar 2', 'Druva', 'Tulsi', 'Pack of 5 leaves', 'Betelnut', 'Banana Leaves with Stem', 'Banana Leaves', 'Pack of 5 Fruits', 'Assorted Flowers']

In [86]:
for k in keys2:
    print (k)

Haar 2
Druva
Tulsi
Pack of 5 leaves
Betelnut
Banana Leaves with Stem
Banana Leaves
Pack of 5 Fruits
Assorted Flowers


In [66]:
df_dropwise.groupby(['drop_point', 'item_name']).sum()

num_packets
drop_point   item_name                           
Ajmera iLand Assorted Flowers                12.0
             Banana Leaves                    6.0
             Banana Leaves with Stem          6.0
             Betelnut                         7.0
             Druva                           20.0
...                                           ...
Sector 1/2   Jhendu/Marigold                  7.0
             Pack of 5 Fruits                10.0
             Pack of 5 leaves                 6.0
             Rajnigandha                      1.0
             Tulsi                            4.0

[62 rows x 1 columns]

In [62]:
for token in range(178, 178+len(to_append)):
    orders_dict[token] = to_append[178-token]